In [1]:
import pickle
import os
import pathlib
from tpot import TPOTClassifier
import sklearn
import pandas as pd
import multiprocessing

In [2]:

partitions = True

if partitions == True:
    print('partitions')
    all_files = [pathlib.Path.cwd() / 'partitions' / 'fms' / ('fm%s.dat' % str(i).zfill(2)) for i in range(0, 61)]
    df = pd.concat((pd.read_pickle(f, compression='gzip') for f in all_files) )  
else:
    print('no partitions')
    df = pd.read_pickle('es0.dat', compression='gzip')

df = df.sample(n=10000, random_state=42)
df['fta'] = df['fta_x'].copy()
df.drop(columns=['fta_x', 'fta_y'], inplace=True)
holdout = df[df.index.str[:4] == '2019'].copy()
df = df[df.index.str[:4] != '2019'].copy()
X = df.reset_index(drop=True)
X = X.fillna(0)
X = X[X.describe().T.index].copy()
y = X.pop("fta")
for feature in X.columns:
    if 'fta' in feature:
        X.drop(labels=feature, axis=1, inplace=True)
for feature in X.columns:
    if '2019' in feature:
        X.drop(labels=feature, axis=1, inplace=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    X.values, y.values, train_size=0.75, test_size=0.25
)


partitions


/home/donald_braman_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [ ]:
pipeline_optimizer = TPOTClassifier(
    generations=0,
    scoring='f1',
    population_size=100,
    verbosity=3,
    n_jobs=multiprocessing.cpu_count()-1,
    config_dict='TPOT light',
)

pipeline_optimizer.fit(X_train, y_train)

19 operators have been imported by TPOT.


In [ ]:
tpot_df = pd.DataFrame.from_dict(data=pipeline_optimizer.evaluated_individuals_, orient='index')
ranked = tpot_df[tpot_df.operator_count==1].sort_values(by='internal_cv_score', ascending=False)
ranked.to_csv('ranked-fta-tt.csv')
print(pipeline_optimizer.score(X_test,y_test))